In [16]:
# !pip install spotipy --upgrade
# !pip install pulp
import spotipy
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import pandas as pd
from IPython.display import clear_output
from pulp import *
import os
from secrets import secrets

In [17]:
# Creating Spotify Connection

## If you want to reproduce this step subscribe to 
## https://developer.spotify.com/dashboard/login, create your app, 
## get your credentials, and then set it as environment variables
cid = secrets['SPOTIPY_CLIEND_ID']
secret = secrets['SPOTIPY_CLIENT_SECRET']

client_credentials_manager = SpotifyClientCredentials(client_id=cid,client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [18]:
# Functon to iterate over paginated results
def get_playlist_tracks(playlist_id):
    results = sp.playlist(playlist_id)['tracks']
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [23]:
tracks = get_playlist_tracks(playlist_id='5TUxgTIxzLbLVh7RUf9V8i')
print("ok")

ok


In [28]:
tr = tracks[0]
sp.audio_features(tr['track']['uri'])[0]

{'danceability': 0.454,
 'energy': 0.91,
 'key': 6,
 'loudness': -7.766,
 'mode': 1,
 'speechiness': 0.0448,
 'acousticness': 0.0866,
 'instrumentalness': 0.0996,
 'liveness': 0.116,
 'valence': 0.629,
 'tempo': 125.116,
 'type': 'audio_features',
 'id': '7o2CTH4ctstm8TNelqjb51',
 'uri': 'spotify:track:7o2CTH4ctstm8TNelqjb51',
 'track_href': 'https://api.spotify.com/v1/tracks/7o2CTH4ctstm8TNelqjb51',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7o2CTH4ctstm8TNelqjb51',
 'duration_ms': 354520,
 'time_signature': 4}

In [29]:
uris = []
track_names = []
artists_name = []
track_pop = []
artists_pop = []
duration = []
danceability = []
energy = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
links = []
for i,track in enumerate(tracks):
    clear_output(wait=True)
    print(f"{round(100*(i+1)/1109,2)}% Done!")
    #URI
    uris.append(track["track"]["uri"])
    
    #Track name
    track_names.append(track["track"]["name"])
    
    #Artist Name
    artists_name.append(track["track"]["artists"][0]["name"])
    
    #Popularity of the track
    track_pop.append(track["track"]["popularity"])

    #Artist Popularity:
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    artists_pop.append(artist_info["popularity"])

    #Track Duration (minutes)
    duration.append(track['track']['duration_ms']/60000)

    #Audio Features:
    features = sp.audio_features(track['track']['uri'])[0]
    danceability.append(features['danceability'])
    energy.append(features['energy'])
    acousticness.append(features['acousticness'])
    instrumentalness.append(features['instrumentalness'])
    liveness.append(features['liveness'])
    valence.append(features['valence'])

    #Creating links
    links.append("open.spotify.com/track/" + uris[i].split(":")[2])

100.0% Done!


In [30]:

df = pd.DataFrame(zip(track_names,artists_name,track_pop,
                      artists_pop,duration,danceability,
                      energy,acousticness,instrumentalness,
                      liveness,valence,links),
                  columns = ["track_name","artist","track_pop",
                            "artist_pop",'duration','danceability',
                            'energy','acousticness','instrumentalness',
                            'liveness','valence','link'])
df.head()

,track_name,artist,track_pop,artist_pop,duration,danceability,energy,acousticness,instrumentalness,liveness,valence,link
0,Sweet Child O' Mine,Guns N' Roses,84,82,5.908667,0.454,0.910,0.0866,0.09960,0.1160,0.629,open.spotify.com/track/7o2CTH4ctstm8TNelqjb51
1,Welcome To The Jungle,Guns N' Roses,80,82,4.533767,0.447,0.954,0.0235,0.40300,0.2980,0.331,open.spotify.com/track/0bVtevEgtDIeRjCJbK3Lmv
2,Paradise City,Guns N' Roses,81,82,6.760667,0.273,0.952,0.0169,0.01110,0.1420,0.472,open.spotify.com/track/3YBZIN3rekqsKxbJc9FZko
3,November Rain,Guns N' Roses,0,82,8.958433,0.197,0.629,0.0165,0.27900,0.1250,0.221,open.spotify.com/track/53968oKecrFxkErocab2Al
4,Knockin' On Heaven's Door,Guns N' Roses,0,82,5.600000,0.486,0.747,0.0203,0.00607,0.0992,0.368,open.spotify.com/track/7gXdAqJLCa5aYUeLVxosOz


In [32]:
# Saving dataframe to File
df.to_csv("../data/for_those_about_to_rock.csv",index=False)